In [1]:
!pip install torchmetrics

In [2]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, GPT2Config
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np

from tabulate import tabulate
from tqdm import trange
import random
from torchmetrics.classification import Recall, Accuracy, AUROC, Precision

In [3]:
FILL = "FILL"

In [4]:
!wget 'https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip'
!unzip -o smsspamcollection.zip

--2024-04-30 03:06:06--  https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘smsspamcollection.zip.4’

smsspamcollection.z     [   <=>              ] 198.65K   295KB/s    in 0.7s    

2024-04-30 03:06:07 (295 KB/s) - ‘smsspamcollection.zip.4’ saved [203415]

Archive:  smsspamcollection.zip
  inflating: SMSSpamCollection       
  inflating: readme                  


In [5]:
!unzip -o smsspamcollection.zip

Archive:  smsspamcollection.zip
  inflating: SMSSpamCollection       
  inflating: readme                  


In [6]:
!head -10 SMSSpamCollection

ham	Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
ham	Ok lar... Joking wif u oni...
spam	Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
ham	U dun say so early hor... U c already then say...
ham	Nah I don't think he goes to usf, he lives around here though
spam	FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, £1.50 to rcv
ham	Even my brother is not like to speak with me. They treat me like aids patent.
ham	As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune
spam	WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.
spam	H

In [7]:
!wget 'https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip'
!unzip -o smsspamcollection.zip

--2024-04-30 03:06:07--  https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘smsspamcollection.zip.5’

smsspamcollection.z     [      <=>           ] 198.65K   139KB/s    in 1.4s    

2024-04-30 03:06:09 (139 KB/s) - ‘smsspamcollection.zip.5’ saved [203415]

Archive:  smsspamcollection.zip
  inflating: SMSSpamCollection       
  inflating: readme                  


In [8]:
file_path = 'SMSSpamCollection'
df = pd.DataFrame({'label':int(), 'text':str()}, index = [])
with open(file_path) as f:
    for line in f.readlines():
        split = line.split('\t')
        df = pd.concat([
                df,
                pd.DataFrame.from_dict({
                    'label': [1 if split[0] == 'spam' else 0],
                    'text': [split[1]]
                })
            ],
            ignore_index=True
        )
df.head()

,label,text
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...\n
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [9]:
text = df.text.values
labels = df.label.values

In [10]:
# Set to the GPT2Tokenizer and set lower case to True
tokenizer = GPT2Tokenizer.from_pretrained('gpt2', add_prefix_space=True)
tokenizer.pad_token = tokenizer.eos_token
# Set the padding to 'left' or 'right'?
# Remember we want to use the last token's embedding to represent the entire sentence
tokenizer.padding_side = 'right'

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [11]:
def print_rand_sentence():
    '''Displays the tokens and respective IDs of a random text sample'''
    index = random.randint(0, len(text)-1)
    print(text[index])
    table = np.array([tokenizer.tokenize(text[index]),
                    tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text[index]))]).T
    print(tabulate(table,
                 headers = ['Tokens', 'Token IDs'],
                 tablefmt = 'fancy_grid'))

print_rand_sentence()

We're all getting worried over here, derek and taylor have already assumed the worst

╒══════════╤═════════════╕
│ Tokens   │   Token IDs │
╞══════════╪═════════════╡
│ ĠWe      │         775 │
├──────────┼─────────────┤
│ 're      │         821 │
├──────────┼─────────────┤
│ Ġall     │         477 │
├──────────┼─────────────┤
│ Ġgetting │        1972 │
├──────────┼─────────────┤
│ Ġworried │        7960 │
├──────────┼─────────────┤
│ Ġover    │         625 │
├──────────┼─────────────┤
│ Ġhere    │         994 │
├──────────┼─────────────┤
│ ,        │          11 │
├──────────┼─────────────┤
│ Ġde      │         390 │
├──────────┼─────────────┤
│ rek      │       37818 │
├──────────┼─────────────┤
│ Ġand     │         290 │
├──────────┼─────────────┤
│ Ġt       │         256 │
├──────────┼─────────────┤
│ aylor    │        7167 │
├──────────┼─────────────┤
│ Ġhave    │         423 │
├──────────┼─────────────┤
│ Ġalready │        1541 │
├──────────┼─────────────┤
│ Ġassumed │        967

In [12]:
token_id = []
attention_masks = []

def preprocessing(input_text, tokenizer):
  '''
  Returns <class transformers.tokenization_utils_base.BatchEncoding> with the following fields:
    - input_ids: list of token ids
    - token_type_ids: list of token type ids
    - attention_mask: list of indices (0,1) specifying which tokens should considered by the model (return_attention_mask = True).
  '''
  # Use the tokenizer and the encode_plus methods to return the right data we'll need
  # Set max_length = 32 and return_tokens = 'pt'
  # Set other fields to the appropriate booleans needed
  return tokenizer.encode_plus(input_text, add_special_tokens=True, max_length=32, truncation=True, padding="max_length", return_attention_mask=True, return_tensors='pt')



for sample in text:
    encoding_dict = preprocessing(sample, tokenizer)
    token_id.append(encoding_dict['input_ids'])
    attention_masks.append(encoding_dict['attention_mask'])


# Gather all the torch_id, attention masks, and labels
token_id = torch.cat(token_id, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels, dtype=torch.long)

In [14]:
def print_rand_sentence_encoding():
    '''Displays tokens, token IDs and attention mask of a random text sample'''
    index = random.randint(0, len(text) - 1)
    tokens = tokenizer.tokenize(tokenizer.decode(token_id[index]))
    print(' '.join(tokens))  # This shows the tokenized text
    token_ids = token_id[index].squeeze().tolist()  # Flatten and convert to list
    attention = attention_masks[index].squeeze().tolist()  # Flatten and convert to list

    # We can only display as many tokens as there are, which is controlled by the actual token length
    # This is because `tokens` do not include padding and will mismatch the length of `token_ids` and `attention`
    num_tokens = len(tokens)
    token_ids = token_ids[:num_tokens]  # Trim to the number of tokens
    attention = attention[:num_tokens]  # Trim to the number of tokens

    # Now create the table with the trimmed lists
    table = np.array(list(zip(tokens, token_ids, attention)))
    print(tabulate(table, headers=['Tokens', 'Token IDs', 'Attention Mask'], tablefmt='fancy_grid'))

print_rand_sentence_encoding()


Ġ ĠThe Ġtable 's Ġoccupied , ĠI 'm Ġwaiting Ġby Ġthe Ġtree Ċ <|endoftext|> <|endoftext|> <|endoftext|> <|endoftext|> <|endoftext|> <|endoftext|> <|endoftext|> <|endoftext|> <|endoftext|> <|endoftext|> <|endoftext|> <|endoftext|> <|endoftext|> <|endoftext|> <|endoftext|> <|endoftext|> <|endoftext|> <|endoftext|> <|endoftext|> <|endoftext|>
╒═══════════════╤═════════════╤══════════════════╕
│ Tokens        │   Token IDs │   Attention Mask │
╞═══════════════╪═════════════╪══════════════════╡
│ Ġ             │         383 │                1 │
├───────────────┼─────────────┼──────────────────┤
│ ĠThe          │        3084 │                1 │
├───────────────┼─────────────┼──────────────────┤
│ Ġtable        │         338 │                1 │
├───────────────┼─────────────┼──────────────────┤
│ 's            │       12030 │                1 │
├───────────────┼─────────────┼──────────────────┤
│ Ġoccupied     │          11 │                1 │
├───────────────┼─────────────┼────────────────

In [15]:
val_ratio = 0.2
# Recommended batch size: 16, 32. See: https://arxiv.org/pdf/1810.04805.pdf
batch_size = 16

# Indices of the train and validation splits stratified by labels
# Use train_test_split
train_idx, val_idx = train_test_split(
    np.arange(len(labels)),
    test_size=val_ratio,
    stratify=labels,
    random_state=42
)

# Train and validation sets
# Set to TensorDataset
train_set = TensorDataset(token_id[train_idx], attention_masks[train_idx], labels[train_idx])

val_set = TensorDataset(token_id[val_idx], attention_masks[val_idx], labels[val_idx])

# Prepare DataLoader
train_dataloader = DataLoader(train_set, batch_size=batch_size, shuffle=True)

validation_dataloader = DataLoader(val_set, batch_size=batch_size)

### Load specific versions of the model

In [17]:
# Load the BertForSequenceClassification model
# Do not ouput the attentions and all hidden states

config = GPT2Config.from_pretrained('gpt2', output_attentions=False, output_hidden_states=False)


# Set to 'gpt2' (the smallest GPT2 which is 120 M parameters)
# Use the config above and set other labels as needed
model = GPT2ForSequenceClassification.from_pretrained('gpt2', config=config)

# Set the pad token id to the eos token id
model.config.pad_token_id = model.config.eos_token_id

# Recommended learning rates (Adam): 5e-5, 3e-5, 2e-5
# See: https://arxiv.org/pdf/1810.04805.pdf
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr = 5e-5,
    eps = 1e-08
)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Set the model to the right device

In [18]:
# If on GPU, do as below
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [19]:
model = model.to(device)

# Recommended number of epochs: See: https://arxiv.org/pdf/1810.04805.pdf
epochs = 2

In [21]:
# Print all the layers of this GPT2 model and the number of parameters per layer
# If this is False, fine tune just the classifier layer and leave all other GPT2 parameters alone
# If this is True, fine tune everything
fine_tune = True

# Turn off gradients using the above
model.requires_grad_(fine_tune) if fine_tune else model.score.weight.requires_grad_(True) and model.score.bias.requires_grad_(True)


assert(total_parameters == 124441344)

### Train the model

In [27]:
from torchmetrics import Accuracy, Recall, Precision, AUROC

# Assuming 'device' is defined (as earlier in your setup), move each metric to the same device as your model
accuracy = Accuracy(num_classes=2, average='macro', task='binary').to(device)
recall = Recall(num_classes=2, average='macro', task='binary').to(device)
precision = Precision(num_classes=2, average='macro', task='binary').to(device)
auroc = AUROC(num_classes=2, task='binary').to(device)


In [28]:
# Main training / validation loop
for _ in trange(epochs, desc = 'Epoch'):

    # ========== Training ==========

    # Set model to training mode
    model.train()

    # Tracking variables
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0

    for step, batch in enumerate(train_dataloader):
        # Put each element of batch onto the device
        batch = tuple(t.to(device) for t in batch)

        # Unpack the batch
        input_ids, attention_mask, labels = batch[0], batch[1], batch[2]

        # Set gradients to zero
        optimizer.zero_grad()

        # Forward pass
        train_output = model(input_ids, attention_mask=attention_mask, labels=labels)

        # Backward pass
        train_output.loss.backward()
        optimizer.step()

        # Update tracking variables
        tr_loss += train_output.loss.item()
        nb_tr_examples += input_ids.size(0)
        nb_tr_steps += 1

    # ========== Validation ==========

    # Set model to evaluation mode
    model.eval()

    # Tracking variables
    val_accuracy = []
    val_precision = []
    val_recall = []
    val_auroc = []

    for batch in validation_dataloader:
        batch = tuple(t.to(device) for t in batch)

        # Unpack the batch
        input_ids, attention_mask, labels = batch[0], batch[1], batch[2]

        with torch.no_grad():
          # Forward pass
            eval_output = model(input_ids, attention_mask=attention_mask)

        # Calculate validation metrics
        labels = labels
        preds = torch.argmax(eval_output.logits, dim=-1)

        val_accuracy.append(accuracy(preds, labels).item())
        val_recall.append(recall(preds, labels).item())
        val_precision.append(precision(preds, labels).item())
        val_auroc.append(auroc(preds, labels.float()).item())  # Ensure labels are appropriate for AUROC calculation


    print('\n\t - Train loss: {:.4f}'.format(tr_loss / nb_tr_steps))
    print('\t - Validation Accuracy: {:.4f}'.format(sum(val_accuracy)/len(val_accuracy)))
    print('\t - Validation Precision: {:.4f}'.format(sum(val_precision)/len(val_precision)))
    print('\t - Validation Recall: {:.4f}'.format(sum(val_recall)/len(val_recall)))
    print('\t - Validation AUROC: {:.4f}\n'.format(sum(val_auroc)/len(val_auroc)))

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torchmetrics/utilities/prints.py:43: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)  # noqa: B028
Epoch:  50%|█████     | 1/2 [00:40<00:40, 40.06s/it]


	 - Train loss: 0.0170
	 - Validation Accuracy: 0.9884
	 - Validation Precision: 0.8476
	 - Validation Recall: 0.8190
	 - Validation AUROC: 0.8656



Epoch: 100%|██████████| 2/2 [01:18<00:00, 39.06s/it]


	 - Train loss: 0.0105
	 - Validation Accuracy: 0.9866
	 - Validation Precision: 0.8571
	 - Validation Recall: 0.7971
	 - Validation AUROC: 0.8557



### Test on a specific sentence, see the outcome

In [29]:
new_sentence = 'WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.'

# We need Token IDs and Attention Mask for inference on the new sentence
test_ids = []
test_attention_mask = []

# Apply the tokenizer
encoding = preprocessing(new_sentence, tokenizer)

# Extract IDs and Attention Mask
test_ids.append(encoding['input_ids'])
test_attention_mask.append(encoding['attention_mask'])
test_ids = torch.cat(test_ids, dim = 0)
test_attention_mask = torch.cat(test_attention_mask, dim = 0)

# Forward pass, calculate logit predictions
with torch.no_grad():
    output = model(test_ids.to(device), token_type_ids = None, attention_mask = test_attention_mask.to(device))

prediction = 'Spam' if np.argmax(output.logits.cpu().numpy()).flatten().item() == 1 else 'Ham'

print('Input Sentence: ', new_sentence)
print('Predicted Class: ', prediction)

Input Sentence:  WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.
Predicted Class:  Spam


### Questions

Question 1: Run the above by fine tuning GPT2 and the classfier head and by not doing this (using GPT2 as a feature encoder). What is the gap between this? What are the metrics we get in each case?

Solution: FILL Paste your output here

Epoch 2/2
Train Loss: 0.0013737583067268133
Train Accuracy: 0.9942917823791504, Train Recall: 0.9689592123031616, Train Precision: 0.9882060885429382, Train AUROC: 0.9835850596427917
Validation Accuracy: 0.9919642857142857, Validation Recall: 0.8547619053295681, Validation Precision: 0.8833333330495017, Validation AUROC: 0.8838744597775595


In [34]:
import torch
from torch.utils.data import DataLoader
from transformers import GPT2ForSequenceClassification, GPT2Config, GPT2Tokenizer
from torchmetrics import Accuracy, Recall, Precision, AUROC
from transformers import AdamW
from tqdm import tqdm
import numpy as np

def train_and_evaluate(fine_tune, train_dataloader, validation_dataloader, device, epochs=2):
    # Initialize tokenizer and model configuration with pad token
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    pad_token_id = tokenizer.eos_token_id  # EOS token is commonly used as pad token.

    config = GPT2Config.from_pretrained('gpt2', pad_token_id=pad_token_id, output_attentions=False, output_hidden_states=False)
    model = GPT2ForSequenceClassification.from_pretrained('gpt2', config=config).to(device)

    if not fine_tune:
        for param in model.parameters():
            param.requires_grad = False
        model.score.weight.requires_grad = True
        model.score.bias.requires_grad = True

    optimizer = AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=5e-5)

    # Metrics
    accuracy = Accuracy(num_classes=2, average='macro', task='binary').to(device)
    recall = Recall(num_classes=2, average='macro', task='binary').to(device)
    precision = Precision(num_classes=2, average='macro', task='binary').to(device)
    auroc = AUROC(num_classes=2, task='binary').to(device)



In [37]:
   # Training loop
for epoch in range(epochs):
        model.train()
        for batch in train_dataloader:
            batch = tuple(t.to(device) for t in batch)
            inputs = {'input_ids': batch[0],
                      'attention_mask': batch[1],
                      'labels': batch[2]}

            optimizer.zero_grad()
            outputs = model(**inputs)
            loss = outputs.loss
            loss.backward()
            optimizer.step()

            preds = torch.argmax(outputs.logits, dim=-1)
            accuracy.update(preds, inputs['labels'])
            recall.update(preds, inputs['labels'])
            precision.update(preds, inputs['labels'])
            auroc.update(preds, inputs['labels'].float())



In [41]:
# Validation loop
model.eval()
val_accuracy, val_recall, val_precision, val_auroc = [], [], [], []
for batch in validation_dataloader:
            batch = tuple(t.to(device) for t in batch)
            inputs = {'input_ids': batch[0],
                      'attention_mask': batch[1],
                      'labels': batch[2]}

            with torch.no_grad():
                outputs = model(**inputs)
                preds = torch.argmax(outputs.logits, dim=-1)
                val_accuracy.append(accuracy(preds, inputs['labels']).item())
                val_recall.append(recall(preds, inputs['labels']).item())
                val_precision.append(precision(preds, inputs['labels']).item())
                val_auroc.append(auroc(preds, inputs['labels'].float()).item())



In [44]:
# Calculate final metrics for this epoch
train_acc, train_recall, train_precision, train_auroc = accuracy.compute(), recall.compute(), precision.compute(), auroc.compute()
accuracy.reset(), recall.reset(), precision.reset(), auroc.reset()
print(f'Epoch {epoch+1}/{epochs}')
print(f'Train Loss: {loss.item()}')
print(f'Train Accuracy: {train_acc}, Train Recall: {train_recall}, Train Precision: {train_precision}, Train AUROC: {train_auroc}')
print(f'Validation Accuracy: {np.mean(val_accuracy)}, Validation Recall: {np.mean(val_recall)}, Validation Precision: {np.mean(val_precision)}, Validation AUROC: {np.mean(val_auroc)}')



Epoch 2/2
Train Loss: 0.0013737583067268133
Train Accuracy: 0.9942917823791504, Train Recall: 0.9689592123031616, Train Precision: 0.9882060885429382, Train AUROC: 0.9835850596427917
Validation Accuracy: 0.9919642857142857, Validation Recall: 0.8547619053295681, Validation Precision: 0.8833333330495017, Validation AUROC: 0.8838744597775595
